# Tiling Sentinel-2 imagery using covering grids

For SamGeo to be able to work effective, it might be worthwhile to tile the different regions using covering grids and generating GeoTIFFs for each subsection. Using smaller tiles might enable SamGeo to be more effective at segmenting field boundaries, while also making visual inspections easier. 

In [1]:
import os
import sys 
sys.path.insert(1, '/Users/rafidmahbub/Desktop/DataKind_Geospatial')
import config
from google.cloud import storage
import ee
from s2cloudless import s2cloudless
from imagery_generation import (
    get_ndvi,
    get_evi,
    spectral_transform
)
import geemap 

## Initialize Google Earth Engine and Cloud Storage

In [2]:
ee.Authenticate()
ee.Initialize(project=config.gee_project)

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './../service_key_gcp.json'
storage_client = storage.Client()

## Create covering grids

For the purposes of tiling, we use the `coveringGrid()` method on `roi`. For example, here we create a $5 \times 5$ grid around the distributor location in Kitale.

In [3]:
coords = ee.Geometry.Point([37.13,-2.1])
roi = coords.buffer(1e4).bounds()
grid = roi.coveringGrid('EPSG:4326', 4500) # covering grid around RoI with sides of 5 km

m = geemap.Map(center=(-2.1,37.13), zoom=12)
m.add_basemap('HYBRID')
m.add_layer(grid)
m

Map(center=[-2.1, 37.13], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

We can modify previous functions for use in image generation. For generating tile images for all distributor locations, we require two loops - 
* the first one loops over the tiles in the covering grid
* the second one loops over each coordinate location

In [24]:
def generate_image_tiles(covering_grid: ee.FeatureCollection, img_type='rgb') -> ee.ImageCollection:
    ''' 
    This function converts an RoI defined by a covering grid of a consisting tiles of a particular size
    and converts them into a series of images in a collection.

    Args: (i) covering_grid - ee.FeatureCollection consisting of rectangles spanning the grid
          (ii) img_type -  argument for the type of image required; img_type can be `rgb`, `false`, `ndvi` and `evi`;
                           defaults to `rgb`
    Returns: ee.ImageCollection() object containing imagery contained within the tiles
    '''
    img_collection = [] # initialize an empty list to store images
    N = covering_grid.size().getInfo()

    for i in range(N):
        tile = covering_grid.getInfo()['features'][i]['geometry']['coordinates']
        tile_AOI = ee.Geometry.Polygon(tile)

        s2_sr_median = s2cloudless(tile_AOI)
        
        if img_type == 'rgb':
            band_list = ['B4', 'B3', 'B2'] # true color band list
            rgb_img = s2_sr_median.select(band_list)
            img_collection.append(rgb_img)

        elif img_type == 'false':
            band_list = ['B8', 'B4', 'B3'] # false color band list
            false_img = s2_sr_median.select(band_list)
            img_collection.append(s2_sr_median.select(false_img))

        elif img_type == 'ndvi':
            ndvi_img = get_ndvi(s2_sr_median).select('NDVI')
            img_collection.append(ndvi_img)

        elif img_type == 'hsv-NDVI':
            hsv_img = spectral_transform(s2_sr_median, 'NDVI')
            img_collection.append(hsv_img)

        elif img_type == 'hsv-EVI':
            hsv_img = spectral_transform(s2_sr_median, 'EVI')
            img_collection.append(hsv_img)

        else:
            raise ValueError(f'Unsupported image type: {img_type}')
        
    return ee.ImageCollection(img_collection)

In [25]:
hsv_img_col = generate_image_tiles(grid, img_type='hsv-EVI')

In [35]:
N = hsv_img_col.size().getInfo()
hsv_img_list = hsv_img_col.toList(N)

vis_params = {'min': 0, 'max': 1.0}

for i in range(N):
    hsv_img = ee.Image(hsv_img_list.get(i))
    coords = grid.getInfo()['features'][i]['geometry']['coordinates']

    hsv_img_vis = hsv_img.visualize(**vis_params)
    export_params = {
            'image': hsv_img_vis,
            'description': 'tile_' + str(i),
            'folder': 'image_tiles',
            'crs': 'EPSG:4326',
            'fileFormat': 'GeoTIFF',
            'scale': 10,
            'region': coords,
            'maxPixels' : 1e9
        }
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()